In [1]:
from glob import glob
from os import mkdir
from os.path import join, exists

from ltbio.biosignals.modalities import EEG

In [2]:
common_path = '/Volumes/MMIS-Saraiv/Datasets/Sapienza/denoised_biosignal'
out_common_path = '/Volumes/MMIS-Saraiv/Datasets/Sapienza/roma_protocol_source_MNE'

In [3]:
# Get recursively all .biosignal files in common_path
all_files = glob(join(common_path, '**/*.biosignal'), recursive=True)

In [4]:
channel_order = ('C3', 'C4', 'Cz', 'F3', 'F4', 'F7', 'F8', 'Fp1', 'Fp2', 'Fpz', 'Fz', 'O1', 'O2', 'P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6')  # without mastoids

In [5]:
for filepath in all_files:
    filename = filepath.split('/')[-1].split('.')[0]
    print(filename)
    subject_out_path = join(out_common_path, filename)
    if not exists(subject_out_path):
        mkdir(subject_out_path)

    subject_out_filepath = join(subject_out_path, 'source_estimates.csv')
    if not exists(subject_out_filepath):

        # Load Biosignal
        x: EEG = EEG.load(filepath)
        x = x[channel_order]  # get only channels of interest
        
        # sLORETA
        sources = x.inverse_solution(method='sLORETA', epoch_duration=2.0, spacing="oct6", mri_space="fsaverage", eeg_channel_locations="/Volumes/MMIS-Saraiv/Datasets/Sapienza/electrodes_auto.xyz")
        sources.to_csv(subject_out_filepath)
